<a href="https://colab.research.google.com/github/benedettacandelori/ADM3_Group6/blob/main/Beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
from google.colab import files

In [4]:
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"benedettacandelori","key":"ac2d50dc84ca8f8bc855d6198827865c"}'}

In [5]:
!cp kaggle.json ~/.kaggle/

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c sapienza-training-camp-2022

sapienza-training-camp-2022.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip sapienza-training-camp-2022

In [9]:
import pandas as pd
import numpy as np
import cv2
# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [10]:
train = pd.read_csv('train.csv')

sample_submission = pd.read_csv('sample_submission.csv')

train.head()

,file_name,category_name,category_id
0,2c24dabe.jpg,motor-yachts,5
1,7dcf4553.jpg,submarines,6
2,33e34b0c.jpg,motor-yachts,5
3,3ac0a11a.jpg,motor-yachts,5
4,f6e59377.jpg,coast-guard,1


In [14]:
train_img = []
for i in range(len(train)):
#for i in range(1):
    img_name = train.loc[i][0] 
    category = train.loc[i][1] 
    
    image_path = 'train/train/' + str(category) + '/' + str(img_name) 
    # reading the image
    img = cv2.imread(image_path)

    img = cv2.resize(img, (256,256))
   
    # normalizing the pixel values
    img = np.divide(img,255.0)
    # converting the type of pixel to float 32
    img = img.astype('float32')
    # appending the image into the list
    train_img.append(img)


# converting the list to numpy array
train_x = np.array(train_img)
# defining the target
train_y = train['category_id'].values
train_x.shape

(2309, 256, 256, 3)

In [ ]:
train_x.shape

In [89]:
train_x[1].shape

(371, 512, 3)

In [15]:
t_train_x, t_val_x, t_train_y,  t_val_y = train_test_split(train_x, train_y, test_size = 0.15)

In [17]:
t_train_x = torch.from_numpy(np.array(t_train_x))
t_train_y = torch.from_numpy(np.array(t_train_y))

In [36]:
from keras.preprocessing.image import image

from keras.preprocessing.image import img_to_array

from tensorflow.keras.applications.resnet50 import preprocess_input

from keras.applications.imagenet_utils import decode_predictions

import matplotlib.pyplot as plt

In [50]:
train_img = []
for i in range(len(train)):
#for i in range(1):
    img_name = train.loc[i][0] 
    category = train.loc[i][1] 
    
    image_path = 'train/train/' + str(category) + '/' + str(img_name) 
    # reading the image
    img = image.load_img(image_path, target_size = (224, 224))
    img = image.img_to_array(img)
    #img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    # appending the image into the list
    train_img.append(img)


# converting the list to numpy array
train_x = np.array(train_img)
# defining the target
train_y = train['category_id'].values
train_x.shape

(2309, 224, 224, 3)

In [51]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50 
model = ResNet50(weights = 'imagenet')

In [53]:
preds = model.predict(train_x, verbose = 1)



73/73 [==============================] - 315s 4s/step


In [ ]:
print('Predicted:', decode_predictions(preds, top=1))

In [61]:
pred_class =[ np.argmax(x) for x in preds]

In [71]:
train['pred_class'] = None
train['pred_class'] = pred_class

In [112]:
cleaned_train = train
wrong_class = []
accepted_classes = [403,484,510,536,540,554,625,628,693,724,814,833]
for i in range(len(train)):
  if train['pred_class'][i] not in accepted_classes:
    cleaned_train = cleaned_train.drop(train.index[i])


In [113]:
cleaned_train.shape

(1902, 4)